In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import os 
import numpy as np
import pandas as pd 
import xarray as xr
from tonic.io import read_config, read_configobj

read configs for filenames and diagnostics to run

In [2]:
# read config files

config_files = read_config(os.path.join(os.getcwd(), 
                                            'filenames.cfg'))
config_diag = read_configobj(os.path.join(os.getcwd(), 
                                       'diagnostics.cfg'))

full list of l2x coupler fields 

In [3]:
l2x_vars = ['l2x_Sl_t', 'l2x_Sl_tref', 'l2x_Sl_qref', 
            'l2x_Sl_avsdr','l2x_Sl_anidr', 'l2x_Sl_avsdf', 
            'l2x_Sl_anidf', 'l2x_Sl_snowh', 'l2x_Sl_u10', 
            'l2x_Sl_fv', 'l2x_Sl_ram1', 'l2x_Sl_logz0', 'l2x_Fall_taux',
            'l2x_Fall_tauy', 'l2x_Fall_lat', 'l2x_Fall_sen', 
            'l2x_Fall_lwup', 'l2x_Fall_evap', 'l2x_Fall_swnet', 
            'l2x_Fall_flxdst1', 'l2x_Fall_flxdst2',
            'l2x_Fall_flxdst3', 'l2x_Fall_flxdst4', 
            'l2x_Flrl_rofliq', 'l2x_Flrl_rofice']

In [4]:
config_diag.items()[0] in config_diag.items()

True

In [7]:
# make plots

# get compset for plot name
if 'RI' in config_files['filenames']['vic4']:
    compset = 'RI'
elif 'RBR' in config_files['filenames']['vic4']:
    compset = 'RBR'
elif 'RB' in config_files['filenames']['vic4']:
    compset = 'RB'
else: 
    raise TypeError("file compset has not yet been defined in the diagnostic script")

# plot settings 
fs = 10
dpi = 50

vic4 = xr.open_dataset(os.path.join(os.getcwd(), 
                                    'coupler_hist_files', 
                                    config_files['filenames']['vic4']))
vic5 = xr.open_dataset(os.path.join(os.getcwd(), 
                                    'coupler_hist_files', 
                                    config_files['filenames']['vic5']))


for plot_type in config_diag.items():
    for l2x_var in l2x_vars:
        if plot_type[0] == "magnitudes" and config_diag['magnitudes']['options']['run']:
            fig, axs = plt.subplots(1, 2, figsize=(15, 4))
            vic4[l2x_var].plot(cmap='viridis', ax=axs[0], add_labels=False)
            axs[0].set_title('VIC 4 %s CPL Hist File' %compset)
            vic5[l2x_var].plot(cmap='viridis', ax=axs[1], add_labels=False)
            axs[1].set_title('VIC 5 %s CPL Hist File' %compset)
        elif (plot_type[0] == "magnitudes_constrain_colorbar" and 
              config_diag['magnitudes_constrain_colorbar']['options']['run']):
            fig, axs = plt.subplots(1, 2, figsize=(15, 4))
            # get colorbar constraints
            if config_diag['magnitudes_constrain_colorbar']['options']['constrain_by'] == "vic4":
                vmin = vic4[l2x_var].min()
                vmax = vic4[l2x_var].max()
            elif config_diag['magnitudes_constrain_colorbar']['options']['constrain_by'] == "vic5":
                vmin = vic5[l2x_var].min()
                vmax = vic5[l2x_var].max()
            else: 
                raise TypeError("this colorbar constrain option is not yet an option, please reset config file")
            vic4[l2x_var].plot(cmap='viridis', ax=axs[0], add_labels=False)
            axs[0].set_title('VIC 4 %s CPL Hist File' %compset)
            vic5[l2x_var].plot(cmap='viridis', ax=axs[1], add_labels=False)
            axs[1].set_title('VIC 5 %s CPL Hist File' %compset)
        elif plot_type[0] == "difference" and config_diag['difference']['options']['run']:
            fig, axs = plt.subplots(1, 1, figsize=(4, 4))
            (vic5[l2x_var] - vic4[l2x_var]).plot(cmap='bwr', ax=axs, add_labels=False)
            axs.set_title('VIC 5 - VIC 4 %s CPL Hist Files' %compset)
        
        plot_direc = os.path.join(os.getcwd(), 'plots', plot_type[0])
        os.makedirs(plot_direc, exist_ok=True)
        plt.suptitle('%s %s' % (l2x_var, vic5[l2x_var].long_name), size=fs)
        plotname = '%s.png' %l2x_var
        savepath = os.path.join(plot_direc, plotname)
        plt.savefig(savepath, format='png', dpi=dpi, bbox_inches='tight')

In [11]:
vic5.l2x_Sl_avsdr.min()

<xarray.DataArray 'l2x_Sl_avsdr' ()>
array(0.0)

In [35]:
if 'RI' in config_files['filenames']['vic4']:
    print("yes")

yes


In [70]:
u = vic4['l2x_Fall_taux']
v = vic5['l2x_Fall_taux']

<xarray.DataArray 'l2x_Fall_taux' (time: 1, l2x_ny: 205, l2x_nx: 275)>
array([[[ nan,  nan, ...,  nan,  nan],
        [ nan,  nan, ...,  nan,  nan],
        ..., 
        [ nan,  nan, ...,  -0.,  -0.],
        [ nan,  nan, ...,  -0.,  -0.]]])
Coordinates:
  * time     (time) datetime64[ns] 1948-09-01
Dimensions without coordinates: l2x_ny, l2x_nx
Attributes:
    units: N m-2
    long_name: Zonal surface stress
    standard_name: surface_downward_eastward_stress
    internal_dname: l2x_lx